In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
base_url = "https://www.casasymas.com.uy/propiedades/venta/casa"

# Páginas a recorrer
paginas = range(1, 60)  
all_links = []

for i in paginas:
    url = f"{base_url}/pagina-{i}"
    print(f"Scrapeando: {url}")
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    # Buscar tarjetas con la clase "card prop-entrada"
    tarjetas = soup.select("a.card.prop-entrada")

    # Extraer los hrefs
    links = [a['href'] for a in tarjetas if a.has_attr("href") and len(a['href']) > 10]
    links = list(set(links))  # eliminar duplicados

    all_links.extend(links)

# Normalizar los links (por si son relativos)
final_links = [f"https://www.casasymas.com.uy{link}" if link.startswith("/") else link for link in all_links]

# Mostrar resultados
print("\n✅ LINKS SCRAPEADOS:")
for link in final_links:
    print(link)
    
print(f"Total: {len(final_links)}")


Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-1
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-2
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-3
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-4
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-5
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-6
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-7
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-8
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-9
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-10
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-11
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-12
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/pagina-13
Scrapeando: https://www.casasymas.com.uy/propie

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

datos = []

for url in final_links:
    print(f"Procesando: {url}")
    try:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")

        # Extraer precio
        precio = soup.select_one(".rate-info")
        precio = precio.get_text(strip=True) if precio else None

        # Extraer características
        caracteristicas = {}
        li_tags = soup.select("div.details-info li")
        
        for li in li_tags:
            img = li.find("img", class_="iconos")
            if img and img.has_attr("title"):
                key = img["title"].strip()
                text = li.get_text(strip=True)
                
                # Quitar el key del texto para obtener solo el valor
                value = text.replace(f"{key}:", "").strip()
                caracteristicas[key] = value

        # Guardar en lista
        datos.append({
            "url": url,
            "precio": precio,
            **caracteristicas
        })
        
        time.sleep(1)  # Pausa para no saturar el sitio

    except Exception as e:
        print(f"❌ Error en {url}: {e}")

# Convertir a DataFrame
df = pd.DataFrame(datos)
print(df.head())

# Guardar a CSV
df.to_csv("propiedades_scrapeadas.csv", index=False)
print("✅ Archivo guardado como propiedades_scrapeadas.csv")


Procesando: https://www.casasymas.com.uy/propiedad/107320-casa-en-alquiler-y-venta-de-3-dormitorios-en-montoya-maldonado
Procesando: https://www.casasymas.com.uy/propiedad/109270-venta-y-alquiler-casa-3-dormitorios-serv-punta-gorda-ciudad-de-guayaquil-en-la-mejor-calle
Procesando: https://www.casasymas.com.uy/propiedad/53906-casa-en-venta-de-3-dormitorios-en-lomas-de-la-tahona-canelones
Procesando: https://www.casasymas.com.uy/propiedad/72905-casa-en-venta-de-2-dormitorios-con-garage-en-la-paloma-rocha
Procesando: https://www.casasymas.com.uy/propiedad/123654-venta-casa-pocitos-4-dormitorios
Procesando: https://www.casasymas.com.uy/propiedad/183832-venta-casa-parque-rivera
Procesando: https://www.casasymas.com.uy/propiedad/134193-venta-casa-de-3-dormitorios-la-tahona-canelones
Procesando: https://www.casasymas.com.uy/propiedad/107333-casa-en-venta-de-4-dormitorios-en-rocha-rocha
Procesando: https://www.casasymas.com.uy/propiedad/153425-casa-en-venta-de-5-dormitorios-en-atlantida-canelo

In [7]:
import pandas as pd

df_csv = pd.read_csv("propiedades_scrapeadas.csv")

In [8]:
df_csv.head(5)

,url,precio,Dormitorios,Baños,Superficie,Superficie Construida,Superficie Balcones,Dormitorios Servicio,Dormitorios Suite,Garage,...,Año de Construcción,Plantas,Piscina,Parrillero,Muebles,Lavadero,Superficie Terreno,Altura,Jardín,Patio
0,https://www.casasymas.com.uy/propiedad/181878-...,U$S 850.000Venta,4,3.0,330 m²,330m²,10m²,Dormitorios de Servicio: 1,Dormitorios en Suite: 1,Garaje: 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.casasymas.com.uy/propiedad/198553-...,U$S 115.000Venta,2,1.0,48 m²,47m²,1m²,NaN,NaN,NaN,...,2009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.casasymas.com.uy/propiedad/144350-...,U$S 175.000Venta,2,1.0,64.5 m²,52m²,6m²,NaN,NaN,Garaje: 1,...,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.casasymas.com.uy/propiedad/162860-...,U$S 140.200Venta,Monoambiente,1.0,42 m²,38m²,5m²,NaN,NaN,NaN,...,NaN,NaN,Piscina,Parrillero,Muebles,Lavadero,NaN,NaN,NaN,NaN
4,https://www.casasymas.com.uy/propiedad/178847-...,U$S 765.000Venta,4,4.0,283 m²,253m²,NaN,Dormitorios de Servicio: 1,Dormitorios en Suite: 1,Garaje: 2,...,1985.0,NaN,NaN,NaN,NaN,Lavadero,NaN,NaN,NaN,NaN
